In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('C:/Users/nguye/Downloads/cs114/data-tbtl/annonimized.csv')

print(data.head())
print(data.info())

            concat('it001',`assignment_id`)  \
0  90ce27571176d87961b565d5ef4b3de33ede04ac   
1  90ce27571176d87961b565d5ef4b3de33ede04ac   
2  90ce27571176d87961b565d5ef4b3de33ede04ac   
3  90ce27571176d87961b565d5ef4b3de33ede04ac   
4  90ce27571176d87961b565d5ef4b3de33ede04ac   

               concat('it001',`problem_id`)  \
0  789454427dd4097a14749e3dde63346b7a8d3811   
1  789454427dd4097a14749e3dde63346b7a8d3811   
2  789454427dd4097a14749e3dde63346b7a8d3811   
3  bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff   
4  7a6e5ca470ff47c3b5048f240c4738de71010c78   

                  concat('it001', username)  is_final status  pre_score  \
0  ed9eaeb6a707f50154024b24d7efcb874a9795dd         0  SCORE          0   
1  ed9eaeb6a707f50154024b24d7efcb874a9795dd         0  SCORE          0   
2  ed9eaeb6a707f50154024b24d7efcb874a9795dd         1  SCORE      10000   
3  ed9eaeb6a707f50154024b24d7efcb874a9795dd         1  SCORE      10000   
4  ed9eaeb6a707f50154024b24d7efcb874a9795dd         1  SCOR

In [3]:
print(data.columns)

Index(['concat('it001',`assignment_id`)', 'concat('it001',`problem_id`)',
       'concat('it001', username)', 'is_final', 'status', 'pre_score',
       'coefficient', 'concat('it001',`language_id`)', 'created_at',
       'updated_at', 'judgement'],
      dtype='object')


In [4]:
data.columns = data.columns.str.replace(r"concat\('it001',\s*", "", regex=True).str.replace(r"\)", "", regex=True).str.strip('`')

print(data.columns)

if 'assignment_id' not in data.columns:
    data.rename(columns={"assignment_id": "assignment_id"}, inplace=True)

if 'username' not in data.columns:
    data.rename(columns={"username": "username"}, inplace=True)

print(data.columns)

data['total_submissions'] = data.groupby('username')['assignment_id'].transform('count')
data['avg_testcase_passed'] = data['pre_score']
data['late_submission_penalty'] = data['coefficient'] * -1

Index(['assignment_id', 'problem_id', 'username', 'is_final', 'status',
       'pre_score', 'coefficient', 'language_id', 'created_at', 'updated_at',
       'judgement'],
      dtype='object')
Index(['assignment_id', 'problem_id', 'username', 'is_final', 'status',
       'pre_score', 'coefficient', 'language_id', 'created_at', 'updated_at',
       'judgement'],
      dtype='object')


In [5]:
tbtl_data = pd.read_excel('C:/Users/nguye/Downloads/cs114/data-tbtl/tbtl-public.ods', engine='odf')

print(tbtl_data.columns)

tbtl_data.rename(columns={"student_id": "username"}, inplace=True) 

data = data.merge(tbtl_data[['username', 'TBTL']], on='username', how='left')

X = data[['total_submissions', 'avg_testcase_passed', 'late_submission_penalty']]  
y = data['TBTL'] 

print(data.head())

Index(['username', 'TBTL'], dtype='object')
                              assignment_id  \
0  90ce27571176d87961b565d5ef4b3de33ede04ac   
1  90ce27571176d87961b565d5ef4b3de33ede04ac   
2  90ce27571176d87961b565d5ef4b3de33ede04ac   
3  90ce27571176d87961b565d5ef4b3de33ede04ac   
4  90ce27571176d87961b565d5ef4b3de33ede04ac   

                                 problem_id  \
0  789454427dd4097a14749e3dde63346b7a8d3811   
1  789454427dd4097a14749e3dde63346b7a8d3811   
2  789454427dd4097a14749e3dde63346b7a8d3811   
3  bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff   
4  7a6e5ca470ff47c3b5048f240c4738de71010c78   

                                   username  is_final status  pre_score  \
0  ed9eaeb6a707f50154024b24d7efcb874a9795dd         0  SCORE          0   
1  ed9eaeb6a707f50154024b24d7efcb874a9795dd         0  SCORE          0   
2  ed9eaeb6a707f50154024b24d7efcb874a9795dd         1  SCORE      10000   
3  ed9eaeb6a707f50154024b24d7efcb874a9795dd         1  SCORE      10000   
4  ed9eaeb6a707

In [6]:
X = data[['total_submissions', 'avg_testcase_passed', 'late_submission_penalty']]
y = data['TBTL']

print("Missing values in y_train:")
print(y.isnull().sum())

y = y.fillna(0)

print("Missing values in y_train after handling:")
print(y.isnull().sum())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print("Model Performance:")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R-squared: {r2_score(y_test, y_pred)}")

Missing values in y_train:
133802
Missing values in y_train after handling:
0
Model Performance:
Mean Absolute Error: 2.1527017922753386
Mean Squared Error: 8.787458543471432
R-squared: 0.45203221531334314


In [7]:
final_predictions = model.predict(X)

submission = pd.DataFrame({'username': data['username'], 'predicted_TBTL': final_predictions})

submission['predicted_TBTL'] = submission['predicted_TBTL'].fillna(0)

submission_unique = submission.drop_duplicates(subset='username', keep='first')

submission_unique.to_csv('predicted_tbtl.csv', index=False)

print(submission_unique.head())

                                     username  predicted_TBTL
0    ed9eaeb6a707f50154024b24d7efcb874a9795dd        1.440736
139  ba12c0a2cb367af0467e479c03507c71a805d291        4.973149
458  0bd2037bf68a97753e5e67ab55dac026a649f279        4.121803
678  b7298b0fe50443a623af9b56792b330c2d052845        4.309537
925  c60be70309789b39355dc612f36e37090ccad5dc        3.562554
